## Option Pricing

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
from scipy.stats import norm
from pypfopt.efficient_frontier import EfficientFrontier as ef
from pypfopt import risk_models as rm
from pypfopt import expected_returns as er

In [2]:
def ticker_frame(tickers, start="2011-01-31", end="2021-02-01", source="yahoo"):
    stock_price = []

    for ticker in range(len(tickers)):
        price_data = pdr.DataReader(tickers[ticker], start=start, end=end, data_source=source)
        stock_price.append(price_data[["Adj Close"]])
    
    df = pd.concat(stock_price, axis=1)
    df.columns=tickers
    return df

In [3]:
def option_pricing(mu, sigma, K, S0, T):
    
    mu = mu
    sigma = sigma
    K = K
    S0 = S0
    T = T
    
    disc = np.exp(-mu*T)
    vol = sigma*np.sqrt(T)
    
    d1 = ((np.log(S0/K) + (mu+0.5*sigma*sigma)*T))/vol
    d2 = d1-vol
    BSprice = S0*norm.cdf(d1)-disc*K*norm.cdf(d2)
    
    N = 1000000
    np.random.seed(0)
    rands = np.random.normal(size=N)
    
    ST = S0*np.exp((mu-0.5*sigma*sigma)*T+vol*rands)
    payoff = np.multiply([p if p > 0 else 0 for p in ST-K], disc)
    MCprice = np.mean(payoff, axis=0)
    
    print("Option price with Black-Scholes and Monte-Carlo are: %f %f" % (BSprice, MCprice))

In [4]:
tickers = ['RBC','CM.TO','TD','BNS','BMO','NA.TO']
df = ticker_frame(tickers)

In [5]:
mu = er.mean_historical_return(df)
sigma = rm.sample_cov(df)

In [6]:
mu

RBC      0.080719
CM.TO    0.086390
TD       0.080989
BNS      0.039446
BMO      0.066392
NA.TO    0.118711
dtype: float64

In [7]:
sigma

,RBC,CM.TO,TD,BNS,BMO,NA.TO
RBC,0.090804,0.027390,0.038154,0.038592,0.038898,0.029170
CM.TO,0.027390,0.033211,0.030022,0.030763,0.031576,0.028397
TD,0.038154,0.030022,0.045035,0.041720,0.041292,0.031964
BNS,0.038592,0.030763,0.041720,0.048817,0.042719,0.033872
BMO,0.038898,0.031576,0.041292,0.042719,0.048462,0.034204
NA.TO,0.029170,0.028397,0.031964,0.033872,0.034204,0.041651


Above we see a covariance matrix. Diagonals of a covariance matrix represent variances.

In [8]:
option_pricing(0.080989, 0.045035, 82.46, 87, 365)

Option price with Black-Scholes and Monte-Carlo are: 87.000000 87.097513
